In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import torch 
import pyro
import inspect

In [14]:
from GenerateDataLM_Examples import make_lm_program_plain, make_lm_program_gamma_gamma, make_lm_program_gamma_gamma_augmented
from LM_abstract import return_only_y

In [15]:
x_test = torch.rand(10, 5)

In [16]:
pprogram1 = make_lm_program_gamma_gamma_augmented()
pprogram1_y = return_only_y(pprogram1)

In [17]:
print(inspect.getsource(pprogram1))

        def multivariate_lm_return_dict(x: torch.tensor, y: torch.tensor = None) -> dict:
                sigma_squared = (pyro.sample("sigma_squared", pyro.distributions.Gamma(a0, b0)) + 0.1)/10000
                
                tau = (pyro.sample("tau", pyro.distributions.Gamma(a1,b1)) + 0.1)/10

                beta_mean = torch.zeros(x.shape[1])
                beta_cov = (1/tau) * torch.eye(x.shape[1])

                beta = pyro.sample("beta", pyro.distributions.MultivariateNormal(beta_mean, beta_cov))

                mean = torch.matmul(x, beta)

                with pyro.plate("data", len(x)):

                        y = pyro.sample("obs", pyro.distributions.Normal(mean, sigma_squared), obs=y)

                return {
                        "x": x,
                        "y": y,
                        "sigma_squared": sigma_squared,
                        "tau": tau,
                        "beta": beta
                }



In [18]:
pprogram1(x_test)

{'x': tensor([[0.4751, 0.9740, 0.6781, 0.8396, 0.0399],
         [0.5208, 0.6352, 0.2324, 0.7731, 0.3670],
         [0.4371, 0.6511, 0.8038, 0.0697, 0.6474],
         [0.8615, 0.3588, 0.5655, 0.1777, 0.2774],
         [0.6581, 0.7667, 0.2053, 0.8710, 0.9429],
         [0.7916, 0.0108, 0.6712, 0.1803, 0.5939],
         [0.6735, 0.8638, 0.4441, 0.8754, 0.1270],
         [0.2292, 0.1116, 0.7377, 0.3345, 0.4324],
         [0.0484, 0.5664, 0.4569, 0.9636, 0.2460],
         [0.2367, 0.0631, 0.3349, 0.1032, 0.8096]]),
 'y': tensor([-2.7189, -1.4626, -2.3615, -3.3263, -1.1892, -1.9007, -2.7987, -0.2675,
          0.2796,  0.2252]),
 'sigma_squared': tensor(0.0003),
 'tau': tensor(0.2493),
 'beta': tensor([-3.3373, -2.2672, -0.6171,  1.7084,  1.4678])}

In [19]:
pprogram1_y(x_test)

tensor([4.7977, 4.7872, 3.3901, 2.9579, 6.7423, 2.8922, 5.1756, 2.3267, 4.2094,
        2.4689])